In [44]:
import json
import pandas as pd

with open('../panel_app/example_data/routing_tomtom_out.json') as f:
    response_json = json.load(f)
list_df_pts = []
leg_list= response_json['routes'][0]['legs']
for i in range(len(leg_list)):
    df = pd.DataFrame(leg_list[i]['points'])
    df['leg'] = i
    list_df_pts.append(df)
df_all_pts = pd.concat(list_df_pts)

DEFAULT_DEST = [
    "1122 3 St SE",
    "1706 11 ave sw",
    "445 Marsh Rd NE",
    "504 Berkley Cres NW",
    "880 Center Ave NE",
    "111 25 Ave SW",
    "7016 34 Ave NW",
    "2313 15A st SE",
]

DEFAULT_LATLONG = [(51.04191, -114.05572),
 (51.04357, -114.0996),
 (51.05276, -114.05288),
 (51.12546, -114.08387),
 (51.05257, -114.04246),
 (51.03027, -114.06479),
 (51.08401, -114.18789),
 (51.03297, -114.02386)]



In [45]:
df_label = pd.DataFrame(data = DEFAULT_LATLONG,columns=['latitude','longitude'])
df_label['label'] = DEFAULT_DEST

In [46]:
import altair as alt
ch = alt.Chart(data = df_all_pts,width = 900,height=600).encode(x=alt.Longitude('longitude',scale=alt.Scale(zero=False)),
                                                                 y=alt.Latitude('latitude',scale=alt.Scale(zero=False)),color = 'leg:N' ).mark_point(filled=True)
ch_lab = alt.Chart(data=df_label,width = 900,height=600).encode(x=alt.Longitude('longitude',scale=alt.Scale(zero=False)),
                                                                 y=alt.Latitude('latitude',scale=alt.Scale(zero=False)),text = 'label:N' ).mark_text(size=15)

(ch+ch_lab).interactive()

alt.LayerChart(...)

In [54]:
from bokeh.plotting import figure, output_notebook, show
from bokeh.tile_providers import CARTODBPOSITRON, get_provider,STAMEN_TERRAIN_RETINA,STAMEN_TERRAIN, Vendors
from bokeh.models import ColumnDataSource
import numpy as np
output_notebook()

Loading BokehJS ...

In [53]:
Vendors

Enumeration(CARTODBPOSITRON, CARTODBPOSITRON_RETINA, STAMEN_TERRAIN, STAMEN_TERRAIN_RETINA, STAMEN_TONER, STAMEN_TONER_BACKGROUND, STAMEN_TONER_LABELS)

In [55]:
from pyproj import Proj, transform

def reproject_coordinates(input_projection, output_projection, input_x, input_y, suffix=' Transformed'):
    """Reproject coordinate series
    Arguments:
        input_projection {string} -- Input Coordinate System descriptor (e.g. 'epsg:4267' for NAD27)
        output_projection {string} -- Output Coordinate System descriptor (e.g. 'epsg:26910' for NAD83)
        input_x {pandas.Series or list-like} -- Array of x values in input CRS
        input_y {pandas.Series or list-like} -- Array of y values in input CRS
    Keyword Arguments:
        suffix {str} -- Appended to original name of input x and y columns (default: {' Transformed'})
    Returns:
        pandas.DataFrame -- Dataframe with two columns: output_x and output_y
    """
    print(f'Converting from {input_projection} to {output_projection}')
    # The 'init=' generates a warning, but transform fails if it is not used!
    output_x, output_y = transform(Proj(init=input_projection, preserve_units=False), Proj(init=output_projection, preserve_units=False), input_x.to_list(), input_y.to_list())
    return pd.DataFrame(data={f'{input_x.name}{suffix}':output_x, f'{input_y.name}{suffix}':output_y})


In [56]:
new_coords=reproject_coordinates('epsg:4326','epsg:3857',df_all_pts['longitude'],df_all_pts['latitude'],suffix='_new')
df_all_pts['Longitude'] = new_coords['longitude_new'].values
df_all_pts['Latitude'] = new_coords['latitude_new'].values

df_source = ColumnDataSource(df_all_pts)

Converting from epsg:4326 to epsg:3857


/home/anton/miniconda3/envs/tf_gpu/lib/python3.7/site-packages/pyproj/crs/crs.py:53: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When making the change, be mindful of axis order changes: https://pyproj4.github.io/pyproj/stable/gotchas.html#axis-order-changes-in-proj-6
  return _prepare_from_string(" ".join(pjargs))
/home/anton/miniconda3/envs/tf_gpu/lib/python3.7/site-packages/pyproj/crs/crs.py:294: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When making the change, be mindful of axis order changes: https://pyproj4.github.io/pyproj/stable/gotchas.html#axis-order-changes-in-proj-6
  projstring = _prepare_from_string(" ".join((projstring, projkwargs)))
/home/anton/miniconda3/envs/tf_gpu/lib/python3.7/site-packages/pyproj/crs/crs.py:53: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the pr

In [59]:
from bokeh.transform import CategoricalColorMapper        
from bokeh.transform import factor_cmap
from bokeh.palettes import Spectral10

df_all_pts['leg'] = df_all_pts['leg'].astype(str)
df_source = ColumnDataSource(df_all_pts)
unique_legs = df_all_pts['leg'].astype(str).unique().tolist()

tile_provider = get_provider(Vendors.STAMEN_TERRAIN_RETINA)
cmap = factor_cmap('leg',palette = Spectral10,
                   factors = sorted(df_all_pts.leg.unique()))
# range bounds supplied in web mercator coordinates
xrange = (df_all_pts['Longitude'].round(decimals=2).min(),
          df_all_pts['Longitude'].round(decimals=2).max())
yrange = (df_all_pts['Latitude'].round(decimals=2).min(),
          df_all_pts['Latitude'].round(decimals=2).max())
p = figure(x_range=xrange,
           y_range=yrange,
            x_axis_type="mercator", y_axis_type="mercator")
p.add_tile(tile_provider)

p.circle(x='Longitude',y ='Latitude',
          #fill_color = cmap,
         color=cmap,
          source =df_source)

p.xaxis.axis_label = 'Longitude'
p.yaxis.axis_label = 'Latitude'

show(p)

In [43]:
import altair as alt
ch = alt.Chart(data = a,width = 900,height=600).encode(x=alt.X('longitude_new',scale=alt.Scale(zero=False)),
                                                                 y=alt.Y('latitude_new',scale=alt.Scale(zero=False))).mark_point(filled=True,size=150,opacity=0.2)
#ch_lab = alt.Chart(data=df_label,width = 900,height=600).encode(x=alt.Longitude('longitude',scale=alt.Scale(zero=False)),
 #                                                                y=alt.Latitude('latitude',scale=alt.Scale(zero=False)),text = 'label:N' ).mark_text(size=15)

(ch).interactive()

alt.Chart(...)

In [25]:
df_all_pts.sample(10)

,latitude,longitude,leg,Longitude,Latitude
106,51.03030,-114.04151,2,-1.269305e+07,6.627027e+06
6,51.12562,-114.08029,6,-1.269716e+07,6.629509e+06
12,51.03714,-114.06855,3,-1.269718e+07,6.628948e+06
244,51.07951,-114.18485,4,-1.269676e+07,6.626793e+06
6,51.05138,-114.05170,8,-1.269716e+07,6.629509e+06
357,51.13936,-114.08486,5,-1.270268e+07,6.629165e+06
24,51.12686,-114.07613,6,-1.269665e+07,6.628588e+06
265,51.15227,-114.15256,5,-1.269711e+07,6.626692e+06
13,51.12650,-114.07864,6,-1.269711e+07,6.628944e+06
97,51.05462,-114.12689,4,-1.269305e+07,6.627480e+06


In [ ]:
from pyproj import Proj, transform

def reproject_coordinates(input_projection, output_projection, input_x, input_y, suffix=' Transformed'):
    """Reproject coordinate series
    Arguments:
        input_projection {string} -- Input Coordinate System descriptor (e.g. 'epsg:4267' for NAD27)
        output_projection {string} -- Output Coordinate System descriptor (e.g. 'epsg:26910' for NAD83)
        input_x {pandas.Series or list-like} -- Array of x values in input CRS
        input_y {pandas.Series or list-like} -- Array of y values in input CRS
    Keyword Arguments:
        suffix {str} -- Appended to original name of input x and y columns (default: {' Transformed'})
    Returns:
        pandas.DataFrame -- Dataframe with two columns: output_x and output_y
    """
    print(f'Converting from {input_projection} to {output_projection}')
    # The 'init=' generates a warning, but transform fails if it is not used!
    output_x, output_y = transform(Proj(init=input_projection, preserve_units=False), Proj(init=output_projection, preserve_units=False), input_x.to_list(), input_y.to_list())
    return pd.DataFrame(data={f'{input_x.name}{suffix}':output_x, f'{input_y.name}{suffix}':output_y})
reproject_coordinates('epsg:4326','epsg:3857',_df['longitude'],_df['latitude'])